In [37]:
%pip install google-genai
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import os
import csv
from google import genai
from google.genai import types 
from dotenv import load_dotenv
import time
import random

In [ ]:
load_dotenv()

client = genai.Client()

In [40]:
def read_questions(csv_path):
    """Read questions into a dict: {question_number: question_text}"""
    questions = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                num, question = row[0].strip(), row[1].strip()
                questions[num] = question
    return questions

In [41]:
def read_options(csv_path):
    """Read options into a dict: {question_number: [option1, option2, ...]}"""
    options = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                num, answer = row[0].strip(), row[1].strip()
                options.setdefault(num, []).append(answer)
    return options

In [42]:
def read_descriptions(csv_path):
    """Read image descriptions into a dict: {question_number: description_text}"""
    descriptions = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                num, desc = row[0].strip(), row[1].strip()
                descriptions[num] = desc

    
    return descriptions

In [ ]:
def ask_gemini_about_description(description, question, options):
    """Ask Gemini about a textual description of an image"""
    
    system_instruction = (
        "You are a specialized **Latvian traffic rules expert**. "
        "Your entire internal reasoning and final output MUST be conducted **solely in English**."
        "---"
        "**ABSOLUTE OUTPUT REQUIREMENT:**"
        "The **ONLY** output you are permitted to generate is the **exact text of the single correct option in English**."
        "You **MUST NOT** include any reasoning, explanation, numbering, or introductory phrases whatsoever."
        "The output **MUST EXACTLY MATCH** one of the provided options and be nothing else."
    )
    
    prompt_text = (
        f"Description: {description}\n\n"
        f"Question: {question}\n"
        f"Options: {options if options else 'No options provided'}"
    )

    contents = [prompt_text]

    config = types.GenerateContentConfig(
        system_instruction=system_instruction,
        max_output_tokens=500,
        thinking_config=types.ThinkingConfig(thinking_budget=0),    
    )

    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=contents,
            config=config,
        )
        
        if response.text is not None:
            return response.text.strip()
        
        if response.candidates and response.candidates[0].finish_reason:
            reason = response.candidates[0].finish_reason.name
            if response.prompt_feedback and response.prompt_feedback.block_reason:
                block_reason = response.prompt_feedback.block_reason.name
                return f"Blocked: {block_reason} (Finish Reason: {reason})"
            return f"Error: Empty response. Finish Reason: {reason}"
        
        return "Error: Empty response (No reason available)."

    except Exception as e:
        return f"API Call Error: {str(e)}"

In [ ]:
def main():
    descriptions_csv = "descriptions.csv"
    questions_csv = "questions.csv"
    options_csv = "options.csv"

    results_all = []

    descriptions = read_descriptions(descriptions_csv)
    questions = read_questions(questions_csv)
    options = read_options(options_csv)
    
    all_nums = sorted(list(set(descriptions.keys()) & set(questions.keys())))

    random.shuffle(all_nums)


    print(f"Processing {len(all_nums)} descriptions in random order.")
    
    count = 1
    for q_num in all_nums:
        if q_num not in descriptions or q_num not in questions:
            continue
            
        description = descriptions[q_num]
        question = questions[q_num]
        answer_options = options.get(q_num, [])

        print(f"Processing question {q_num}... ({count}/{len(all_nums)})")
        
        answer = ask_gemini_about_description(description, question, answer_options) 
        
        count += 1

        results_all.append({
            "image_number": q_num,
            "model_answer": answer
        })

        print(f"Question {q_num}: {answer}\n")

    output_csv_file = "results_Gemini_description.csv"
    with open(output_csv_file, "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["image_number", "question", "options", "model_answer"]
        )
        writer.writeheader()
        for result in results_all:
            q_num = result["image_number"]
            
            writer.writerow({
                "image_number": q_num,
                "question": questions.get(q_num, ""),
                "options": ";".join(options.get(q_num, [])),
                "model_answer": result["model_answer"]
            })

    print(f"Saved all results to: {output_csv_file}")

In [45]:
if __name__ == "__main__":
    main()

Processing 100 descriptions in random order.
Processing question 37... (1/100)
Question 37: That road works are being carried out on the road.

Processing question 19... (2/100)
Question 19: A.

Processing question 93... (3/100)
Question 93: I will cross first.

Processing question 7... (4/100)
Question 7: You will give way to the cyclist.

Processing question 79... (5/100)
Question 79: Cycleway will cross the carriageway.

Processing question 87... (6/100)
Question 87: I will give way to the cyclist.

Processing question 17... (7/100)
Question 17: A B.

Processing question 20... (8/100)
Question 20: A place where parking is prohibited.

Processing question 62... (9/100)
Question 62: To all.

Processing question 22... (10/100)
Question 22: You will give way to both car drivers.

Processing question 31... (11/100)
Question 31: You will give way to both drivers.

Processing question 89... (12/100)
Question 89: It is allowed.

Processing question 55... (13/100)
Question 55: Both vehicles 